# 社交媒体评论点赞预测系统

## 项目概述

这是一个基于机器学习的社交媒体评论点赞数预测系统，专门针对B站和小红书两大平台的用户评论进行分析和预测。该系统使用随机森林回归算法，通过分析评论的各种特征（如评论长度、发布时间、是否含表情等），预测评论可能获得的点赞数量，为内容创作者和社交媒体营销人员提供数据支持。

## 核心功能

1. **数据处理和特征工程**
   - 支持统一处理B站和小红书的评论数据
   - 自动提取评论特征（评论长度、表情使用、发布时段等）
   - 针对不同平台提取特定特征（B站用户等级、小红书IP地址等）

2. **模型训练与管理**
   - 训练新模型或基于现有模型继续训练
   - 自动评估模型性能（MSE、R²评分）
   - 可视化展示特征重要性和预测效果
   - 模型版本管理和时间戳命名

3. **点赞数预测**
   - 基于训练好的模型预测评论点赞潜力
   - 支持选择不同模型进行预测
   - 根据平台差异动态调整输入特征
   - 记录和导出预测结果

## 技术特点

- **数据处理**：使用pandas进行高效数据处理和转换
- **机器学习**：基于scikit-learn的RandomForestRegressor实现回归预测
- **可视化**：利用matplotlib和seaborn展示模型性能和特征重要性
- **用户界面**：基于ipywidgets构建交互式Jupyter界面，支持直观操作
- **错误处理**：完善的错误提示和异常处理机制

## 使用场景

1. **内容创作优化**：帮助创作者预测评论获赞潜力，优化表达方式
2. **社交媒体营销**：为营销人员提供评论效果预测，制定更有效的互动策略
3. **用户行为研究**：分析不同平台用户的点赞行为特征和偏好
4. **内容运营决策**：为平台运营提供数据支持，了解用户互动模式

## 项目特色

- **跨平台分析**：同时支持B站和小红书两个主流平台的数据分析
- **特征差异化**：根据不同平台的特点提取和处理特定特征
- **模型可扩展性**：支持模型迭代更新和多版本管理
- **用户友好界面**：直观的操作界面，降低使用门槛
- **可视化洞察**：提供丰富的可视化结果，帮助理解模型预测逻辑

通过这个系统，用户可以深入了解影响评论点赞数的关键因素，预测评论的互动潜力，从而制定更有效的内容策略和互动方案。

# 数据处理和特征工程代码块分析

第一个代码块实现了从原始Excel文件到结构化数据集的转换过程，主要完成以下功能：

## 1. 数据导入与整合
- 引入必要的库：`os`, `pandas`, `re`
- 分别读取B站和小红书的Excel评论数据文件
- 处理不同格式的数据源，统一字段名和数据结构

## 2. 平台特定数据处理
- **B站数据处理**:
  - 提取帖子标题信息
  - 将"一级评论"、"二级评论"等文本转换为数字标识
  - 处理用户性别、等级等特有字段
  - 计算二级回复数量作为互动指标

- **小红书数据处理**:
  - 统一时间格式
  - 处理IP地址信息
  - 提取评论关系和二级回复数据

## 3. 特征工程
- 创建评论字数统计特征
- 识别评论中的表情符号 `[xxx]`
- 对B站数据，处理发布时段（早/中/晚）
- 对用户等级进行分类（资深用户/普通用户）
- 创建互动值指标：B站使用二级评论数，小红书使用二级评论数/10

## 4. 数据保存与输出
- 合并处理后的数据集
- 使用时间戳命名并保存为Excel文件
- 输出处理统计信息和样本数据

这段代码展示了针对社交媒体评论数据的专业ETL（提取-转换-加载）流程，为后续的机器学习模型训练提供了结构化、标准化的数据集。从处理结果可以看出，最终合并的数据集包含750条记录和12个字段，为评论点赞预测奠定了基础。

In [10]:
import os
import pandas as pd
import re

def unify_data(folder_bili='data/BILI', folder_xhs='data/XHS'):
    standard_cols = [
        'user_nickname','comment_text','replied_user','comment_level',
        'likes','reply_time','gender','user_level','ip','second_level_reply_count'
    ]

    # 处理 B站数据
    bili_dfs = []
    for file in os.listdir(folder_bili):
        if file.endswith('.xlsx'):
            try:
                print(f"处理B站文件: {file}")
                # 先读取第一行最后一列以获取帖子标题
                temp = pd.read_excel(os.path.join(folder_bili, file), header=None)
                post_title = str(temp.iloc[0, -1])
                # 实际数据从第二行开始，但不使用列名，因为没有标题行
                df = pd.read_excel(os.path.join(folder_bili, file), skiprows=1, header=None)
                
                # 打印列内容以便调试
                print(f"B站文件列内容示例: {df.iloc[0].tolist()}")
                
                # 根据位置直接指定列名 - 根据示例数据推断
                column_names = {
                    0: 'user_nickname',  # 第1列: 用户昵称
                    1: 'gender',         # 第2列: 性别
                    2: 'comment_text',   # 第3列: 评论内容
                    3: 'replied_user',   # 第4列: 被回复用户
                    4: 'comment_level_text', # 第5列: 评论层级描述
                    5: 'user_level',     # 第6列: 用户等级
                    6: 'likes',          # 第7列: 点赞数
                    7: 'reply_time',     # 第8列: 回复时间
                }
                
                # 重命名列
                df = df.rename(columns=column_names)
                
                # 转换评论层级文本为数字: "一级评论" -> 1, "二级评论" -> 2
                if 'comment_level_text' in df.columns:
                    df['comment_level'] = df['comment_level_text'].apply(
                        lambda x: 1 if x == '一级评论' else (2 if x == '二级评论' else 1)
                    )
                else:
                    df['comment_level'] = 1
                
                # 计算二级回复数
                second_level_counts = df[df['comment_level'] == 2].groupby('replied_user').size()
                df['second_level_reply_count'] = df.apply(
                    lambda row: second_level_counts.get(row['user_nickname'], 0) 
                    if row['comment_level'] == 1 else None, 
                    axis=1
                )
                
                # 确保所有必要列存在
                for col in standard_cols:
                    if col not in df.columns:
                        df[col] = None
                        
                df['ip'] = None
                df['post_title'] = post_title
                df['source'] = 'B站'
                df = df.reindex(columns=standard_cols + ['post_title','source'])
                bili_dfs.append(df)
            except Exception as e:
                print(f"处理B站文件 {file} 时出错: {str(e)}")
                import traceback
                traceback.print_exc()
                
    bili_all = pd.concat(bili_dfs, ignore_index=True) if bili_dfs else pd.DataFrame(columns=standard_cols + ['post_title','source'])

    # 处理小红书数据
    xhs_dfs = []
    for file in os.listdir(folder_xhs):
        if file.endswith('.xlsx'):
            try:
                print(f"处理小红书文件: {file}")
                # 先读取第一行最后一列以获取帖子标题
                temp = pd.read_excel(os.path.join(folder_xhs, file), header=None)
                post_title = str(temp.iloc[0, -1])
                # 实际数据从第二行开始，同样可能没有标题行
                df = pd.read_excel(os.path.join(folder_xhs, file), skiprows=1, header=None)
                
                # 打印列内容以便调试
                print(f"小红书文件列内容示例: {df.iloc[0].tolist() if not df.empty else '空文件'}")
                
                # 根据位置直接指定列名 - 根据常见结构推断
                try:
                    column_names = {
                        0: 'user_nickname',  # 第1列: 用户昵称
                        1: 'comment_text',   # 第2列: 评论内容
                        2: 'replied_user',   # 第3列: 被回复用户
                        3: 'comment_level_text', # 第4列: 评论层级描述
                        4: 'ip',             # 第5列: IP地址
                        5: 'likes',          # 第6列: 点赞数
                        6: 'second_level_reply_count', #第7列：二级回复数
                        7: 'reply_time',     # 第8列: 时间
                        
                    }
                    
                    # 重命名列
                    df = df.rename(columns=column_names)
                    
                    # 默认所有小红书评论为二级评论，除非能确定层级
                    df['comment_level'] = 2
                    # 如果 replied_user 有值，则可能是一级评论
                    if 'replied_user' in df.columns:
                        df.loc[df['second_level_reply_count'].notna() & (df['second_level_reply_count'] != ''), 'comment_level'] = 1
                except Exception as e:
                    print(f"小红书列名映射错误: {e}")
                    # 如果出错，使用更保守的方法映射列
                    actual_cols = df.columns.tolist()
                    for i, col in enumerate(actual_cols):
                        if i < len(['user_nickname', 'comment_text', 'replied_user', 'likes', 'reply_time', 'ip']):
                            df = df.rename(columns={col: ['user_nickname', 'comment_text', 'replied_user', 'likes', 'reply_time', 'ip'][i]})
                    df['comment_level'] = 1
                
                # 确保所有必要列存在
                for col in standard_cols:
                    if col not in df.columns:
                        df[col] = None
                
                # 将小红书时间从各种格式转换为标准格式
                if 'reply_time' in df.columns and df['reply_time'].notna().any():
                    try:
                        df['reply_time'] = pd.to_datetime(df['reply_time'], errors='coerce').dt.strftime('%Y-%m-%d')
                    except:
                        print("警告: 小红书时间格式转换失败")
                
                df['gender'] = None 
                df['user_level'] = None
                df['post_title'] = post_title
                df['source'] = '小红书'
                df = df.reindex(columns=standard_cols + ['post_title','source'])
                xhs_dfs.append(df)
            except Exception as e:
                print(f"处理小红书文件 {file} 时出错: {str(e)}")
                import traceback
                traceback.print_exc()
                
    xhs_all = pd.concat(xhs_dfs, ignore_index=True) if xhs_dfs else pd.DataFrame(columns=standard_cols + ['post_title','source'])

    # 合并数据
    combined_df = pd.concat([bili_all, xhs_all], ignore_index=True)
    print(f"合并数据集大小: {combined_df.shape}")
    return combined_df

def feature_engineering(df):
    try:
        # 评论字数统计
        df['comment_length'] = df['comment_text'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
        
        # 是否带表情（匹配 [xxx]）
        df['has_emoji'] = df['comment_text'].apply(
            lambda x: '是' if pd.notnull(x) and re.search(r'\[[^]]*\]', str(x)) else '否'
        )
        
        # B站评论发布时段：早/中/晚（只对 B站有效）
        df['time_slot'] = None
        mask_bili = (df['source'] == 'B站') & df['reply_time'].notnull()
        
        # 确保B站时间数据是datetime格式
        if mask_bili.any():
            try:
                # 先确保时间列是字符串
                df.loc[mask_bili, 'reply_time'] = df.loc[mask_bili, 'reply_time'].astype(str)
                # 尝试转换为datetime
                df.loc[mask_bili, 'reply_time_dt'] = pd.to_datetime(df.loc[mask_bili, 'reply_time'], errors='coerce')
                
                # 基于转换后的datetime确定时段
                df.loc[mask_bili, 'time_slot'] = df.loc[mask_bili, 'reply_time_dt'].apply(
                    lambda t: '早' if pd.notnull(t) and t.hour < 12 
                            else ('中' if pd.notnull(t) and t.hour < 18 
                            else ('晚' if pd.notnull(t) else None))
                )
                # 删除临时列
                df.drop('reply_time_dt', axis=1, inplace=True, errors='ignore')
            except Exception as e:
                print(f"时间段处理错误: {e}")
                import traceback
                traceback.print_exc()
        
        # 用户等级转换（仅 B站）
        df['user_level_converted'] = None
        try:
            # 确保用户等级是数值型
            mask_bili_level = (df['source'] == 'B站') & df['user_level'].notnull()
            if mask_bili_level.any():
                df.loc[mask_bili_level, 'user_level'] = pd.to_numeric(df.loc[mask_bili_level, 'user_level'], errors='coerce')
                
                df['user_level_converted'] = df.apply(
                    lambda row: '资深用户' if (row['source'] == 'B站' 
                                            and pd.notnull(row['user_level']) 
                                            and pd.to_numeric(row['user_level'], errors='coerce') >= 4)
                    else ('普通用户' if (row['source'] == 'B站' and pd.notnull(row['user_level'])) else None),
                    axis=1
                )
        except Exception as e:
            print(f"用户等级转换错误: {e}")
            import traceback
            traceback.print_exc()
        
        # 互动值：小红书二级评论数/10；B站二级评论数
        df['interaction_score'] = 0  # 默认值设为0
        try:
            # 安全转换second_level_reply_count为数值
            df['second_level_reply_count_num'] = pd.to_numeric(df['second_level_reply_count'], errors='coerce').fillna(0)
            
            # B站互动值就是二级评论数
            mask_bili = df['source'] == 'B站'
            df.loc[mask_bili, 'interaction_score'] = df.loc[mask_bili, 'second_level_reply_count_num']
            
            # 小红书互动值是二级评论数/10
            mask_xhs = df['source'] == '小红书'
            df.loc[mask_xhs, 'interaction_score'] = df.loc[mask_xhs, 'second_level_reply_count_num'] / 10
            
            # 删除临时列
            df.drop('second_level_reply_count_num', axis=1, inplace=True, errors='ignore')
        except Exception as e:
            print(f"互动值计算错误: {e}")
            import traceback
            traceback.print_exc()
            
        return df
    except Exception as e:
        print(f"特征工程失败: {e}")
        import traceback
        traceback.print_exc()
        return df

# 调用示例
try:
    data_all = unify_data(folder_bili='data/BILI', folder_xhs='data/XHS')
    data_all = feature_engineering(data_all)
    # 避免同一文件的写入冲突
    output_filename = "final_data_with_features_" + pd.Timestamp.now().strftime("%Y%m%d_%H%M%S") + ".xlsx"
    data_all.to_excel(output_filename, index=False)
    print(f"数据已保存到 {output_filename}")
except Exception as e:
    print(f"处理失败: {e}")
    import traceback
    traceback.print_exc()

处理B站文件: comment_output.xlsx
B站文件列内容示例: ['小杨Johnson', '男', '小羊村还得是小杨来[害羞]', nan, '一级评论', np.int64(6), np.int64(6179), '2025-03-21 23:03:27', np.float64(nan)]
处理B站文件: comment_output22.xlsx
B站文件列内容示例: ['小杨Johnson', '男', '小羊村还得是小杨来[害羞]', nan, '一级评论', np.int64(6), np.int64(6179), '2025-03-21 23:03:27', np.float64(nan)]
处理B站文件: comment_output_1.xlsx
B站文件列内容示例: ['三与三十万患者', '男', '00:11 可爱捏', nan, '一级评论', np.int64(5), np.int64(18), '2025-03-18 11:01:58', np.float64(nan)]
处理B站文件: comment_output_10.xlsx
B站文件列内容示例: ['双持冰淇淋', '保密', '视频里说喜欢的你人一般不会评论，那就评论下吧………\n19年的老粉丝，并且经常看直播，表示大祥哥这人就是拧巴、爱面儿、爱得瑟、藏不住事，但确实很真诚，简直就是骗子眼里最完美的被骗对象[喜极而泣]', nan, '一级评论', np.int64(6), np.int64(12015), '2025-03-25 21:24:47', np.float64(nan)]
处理B站文件: comment_output_11.xlsx
B站文件列内容示例: ['漫行星', '保密', '汽烧都很难，别说用柴烧了', nan, '一级评论', np.int64(5), np.int64(38), '2025-03-27 00:32:39', np.float64(nan)]
处理B站文件: comment_output_12.xlsx
B站文件列内容示例: ['一条卓也', '保密', '该下手她是真下手[笑哭]', nan, '一级评论', np.int64(5), np.int64(123), '2025-03-28 14:00:56', np

C:\Users\33223\AppData\Local\Temp\ipykernel_28924\121182624.py:217: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 0.7  0.7  0.8  0.2  0.   0.   0.   0.3  1.   0.   0.1  0.1  0.3  0.1
  0.  11.1  0.   3.3  0.   1.7  0.   0.   0.   1.1  0.   0.   0.   0.5
  0.   0.   0.2  0.   0.   0.   0.   0.   0.1  0.   0.   0.   0.   0.
  0.   0.4  0.   0.4  0.   0.4  0.   0.   0.   0.1  0.   0.   0.   0.2
  0.   0.   0.2  0.   0.2  0.   0.   0.1  0.   0.   0.1  0.   0.1  0.4
  0.   0.8  0.   0.4  0.   0.7  0.   0.9  0.   0.7  0.   0.3  0.   0.6
  0.   0.6  0.   0.2  0.  11.1  0.   3.3  0.   1.7  0.   0.   0.   1.1
  0.   0.   0.   0.5  0.   0.   0.7  0.7  0.8  0.2  0.   0.   0.   0.3
  1.   0.   0.1  0.1  0.3  0.1  0.   0.4  0.   0.8  0.   0.4  0.   0.7
  0.   0.9  0.   0.7  0.   0.3  0.   0.6  0.   0.6  0.   0.2  0. ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[m

数据已保存到 final_data_with_features_20250401_170728.xlsx


# 社交媒体评论点赞数预测系统 - 技术概述

## 项目架构与功能设计

这是一个基于机器学习的评论数据分析系统，专注于B站和小红书两大平台的用户评论点赞行为建模。系统采用随机森林回归算法，通过多维特征提取与分析，构建点赞数量预测模型，为内容创作者和社交媒体运营提供数据驱动的决策支持。

### 核心技术框架

- **数据处理引擎**：基于Pandas的ETL流程，实现跨平台数据整合与标准化
- **特征工程模块**：自动提取文本特征、时间特征和用户特征，支持平台特定特征差异化处理
- **ML预测引擎**：基于RandomForestRegressor的集成学习模型，支持增量训练与模型更新
- **异常值处理**：基于百分位数的自适应过滤机制，提高模型稳定性和泛化能力
- **交互式界面**：基于ipywidgets的选项卡式响应界面，支持动态表单调整和实时预测

### 数据流与处理管道

1. **数据获取与初步清洗**
   - 自动识别B站/小红书数据格式并进行统一标准化
   - 智能处理缺失值、格式转换和字段映射
   - 统一评论层级体系和时间戳格式

2. **特征抽取与转换**
   - 多维度特征工程：文本长度、表情符号识别、时段分析、用户等级映射
   - 平台差异化特征：B站(发布时段、用户等级)、小红书(IP地址关联)
   - 互动价值量化：基于二级评论数的互动价值评估指标构建

3. **模型训练与优化**
   - 支持新模型训练和已有模型增量更新
   - 自动过滤极端值提高模型稳定性，可配置过滤阈值
   - 特征重要性评估与可视化，提供模型解释性
   - 模型性能指标监控(MSE、R²)和结果可视化

4. **预测与结果输出**
   - 实时评论点赞潜力评估
   - 根据不同平台动态调整预测参数
   - 提供预测结果记录与导出
   - 支持模型对比和预测追溯

## 系统亮点与技术价值

- **智能数据适配**：自动识别并适配两大平台不同的数据格式与特征结构
- **异常值自适应处理**：通过可配置的百分位阈值过滤极端点赞数据，有效提升模型准确性
- **特征差异化处理**：针对不同平台的特性进行个性化特征工程和预测流程
- **模型版本管理**：支持时间戳命名和模型迭代，跟踪模型演化过程
- **细粒度特征重要性分析**：提供详细的特征影响力分析，指导内容优化策略

通过整合数据科学和人机交互技术，该系统为社交媒体内容创作和运营提供了数据驱动的分析工具，支持从数据预处理到模型训练再到点赞预测的全流程应用场景，帮助用户在复杂多变的社交媒体环境中做出更明智的内容决策。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
import pickle
import os
from datetime import datetime
import glob

# 确保models文件夹存在
if not os.path.exists('models'):
    os.makedirs('models')
    print("已创建models文件夹")

# 1. 导入数据集并进行预处理
def load_data(file_path):
    """加载数据集并进行基本预处理"""
    print(f"正在加载数据: {file_path}")
    
    # 检查文件路径是否为空
    if not file_path or file_path.strip() == '':
        print("错误: 文件路径不能为空")
        print("请输入有效的数据文件路径")
        return None
    
    # 检查文件路径是否有效
    if not os.path.exists(file_path):
        print(f"错误: 找不到文件 '{file_path}'")
        print("请确认文件路径是否正确，并确保文件已保存在指定位置。")
        print(f"当前工作目录: {os.getcwd()}")
        return None
    
    try:
        df = pd.read_excel(file_path)
        
        # 显示基本信息
        print(f"数据集大小: {df.shape}")
        print("\n数据集列:")
        for col in df.columns:
            print(f"- {col}: {df[col].dtype}")
        
        return df
    except Exception as e:
        print(f"读取文件时出错: {str(e)}")
        print("提示: 请确保文件格式正确且未被其他程序占用。")
        return None

# 获取所有可用模型
def get_available_models():
    """获取models文件夹中所有可用的.pkl模型文件"""
    model_files = glob.glob('models/*.pkl')
    return [os.path.basename(f) for f in model_files]

# 加载保存的模型
def load_model(model_name):
    """从models文件夹加载指定的模型"""
    try:
        model_path = os.path.join('models', model_name)
        with open(model_path, 'rb') as f:
            return pickle.load(f)
    except Exception as e:
        print(f"加载模型时出错: {str(e)}")
        return None

# 2. 数据预处理和特征工程
def preprocess_data(df):
    """对数据进行预处理和特征工程"""
    # 复制一份数据避免修改原始数据
    df_processed = df.copy()
    
    # 转换分类特征为数值型
    if 'gender' in df_processed.columns:
        gender_map = {'男': 1, '女': 2, '保密': 0}
        df_processed['gender_num'] = df_processed['gender'].map(gender_map).fillna(0)
    
    # 转换时间为数值特征
    if 'reply_time' in df_processed.columns:
        df_processed['reply_time'] = pd.to_datetime(df_processed['reply_time'], errors='coerce')
        df_processed['reply_hour'] = df_processed['reply_time'].dt.hour.fillna(-1)
        df_processed['reply_dayofweek'] = df_processed['reply_time'].dt.dayofweek.fillna(-1)
    
    # 转换时间段为数值
    if 'time_slot' in df_processed.columns:
        time_slot_map = {'早': 0, '中': 1, '晚': 2}
        df_processed['time_slot_num'] = df_processed['time_slot'].map(time_slot_map).fillna(-1)
    
    # 转换来源为数值
    if 'source' in df_processed.columns:
        source_map = {'B站': 0, '小红书': 1}
        df_processed['source_num'] = df_processed['source'].map(source_map).fillna(-1)
    
    # 转换表情特征
    if 'has_emoji' in df_processed.columns:
        df_processed['has_emoji_num'] = df_processed['has_emoji'].map({'是': 1, '否': 0}).fillna(0)
    
    # 确保点赞数是数值型
    if 'likes' in df_processed.columns:
        df_processed['likes'] = pd.to_numeric(df_processed['likes'], errors='coerce').fillna(0)
    
    # 用户等级转换为数值
    if 'user_level_converted' in df_processed.columns:
        level_map = {'资深用户': 1, '普通用户': 0}
        df_processed['user_level_converted_num'] = df_processed['user_level_converted'].map(level_map).fillna(-1)
    
    # 处理缺失值
    numeric_features = ['comment_length', 'likes', 'interaction_score']
    for feature in numeric_features:
        if feature in df_processed.columns:
            df_processed[feature] = pd.to_numeric(df_processed[feature], errors='coerce').fillna(0)
    
    print("预处理完成!")
    return df_processed

# 3. 模型训练
def train_model(df, test_size=0.2, random_state=42, existing_model=None, filter_outliers=True, percentile_threshold=99):
    """训练随机森林模型并返回模型和评估指标"""
    plt.rc("font", family='MicroSoft YaHei', weight="bold")
    
    # 选择特征列，去除不适合作为特征的列
    features = [col for col in df.columns if col.endswith('_num') or 
                col in ['comment_length', 'interaction_score', 'reply_hour', 'reply_dayofweek']]
    
    # 确保所有特征都存在
    features = [f for f in features if f in df.columns]
    
    print(f"使用以下特征进行训练: {features}")
    
    # 过滤异常值
    original_size = len(df)
    if filter_outliers and 'likes' in df.columns:
        # 显示点赞数分布情况
        print("\n点赞数统计:")
        print(f"- 最小值: {df['likes'].min()}")
        print(f"- 最大值: {df['likes'].max()}")
        print(f"- 平均值: {df['likes'].mean():.2f}")
        print(f"- 中位数: {df['likes'].median()}")
        
        # 计算百分位数阈值
        likes_threshold = df['likes'].quantile(percentile_threshold/100)
        print(f"- {percentile_threshold}百分位数: {likes_threshold}")
        
        # 过滤极端点赞数
        filtered_df = df[df['likes'] <= likes_threshold].copy()
        
        # 报告过滤情况
        filtered_size = len(filtered_df)
        filtered_count = original_size - filtered_size
        
        print(f"\n已过滤 {filtered_count} 条异常点赞数据 ({filtered_count/original_size*100:.2f}%)")
        print(f"过滤后数据集大小: {filtered_size}")
        
        # 使用过滤后的数据集
        df = filtered_df
        
        # 绘制过滤前后的点赞数分布
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        sns.histplot(df['likes'], kde=True)
        plt.title('过滤后点赞数分布')
        plt.xlabel('点赞数')
        
        plt.subplot(1, 2, 2)
        sns.boxplot(x=df['likes'])
        plt.title('过滤后点赞数箱线图')
        plt.xlabel('点赞数')
        
        plt.tight_layout()
        plt.show()
    
    X = df[features]
    y = df['likes']
    
    # 拆分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # 训练随机森林模型
    if existing_model is not None:
        print(f"使用已有模型继续训练")
        model = existing_model
        # 继续训练现有模型
        model.n_estimators += 50  # 增加更多树
        model.fit(X_train, y_train)
    else:
        print(f"训练新模型")
        model = RandomForestRegressor(n_estimators=100, random_state=random_state)
        model.fit(X_train, y_train)
    
    # 预测和评估
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"模型训练完成:")
    print(f"- 均方误差 (MSE): {mse:.2f}")
    print(f"- R² 评分: {r2:.2f}")
    
    # 特征重要性
    feature_importance = pd.DataFrame({
        'Feature': features,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\n特征重要性:")
    display(feature_importance)
    
    # 绘制特征重要性图
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title('特征重要性')
    plt.tight_layout()
    plt.show()
    
    # 绘制预测值与实际值对比散点图
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
    plt.xlabel('实际点赞数')
    plt.ylabel('预测点赞数')
    plt.title('实际点赞数 vs 预测点赞数')
    plt.tight_layout()
    plt.show()
    
    return model, feature_importance
# 4. 创建用于预测的函数
def prepare_input_for_prediction(input_data, feature_names):
    """准备输入数据用于预测"""
    # 创建包含所有特征的DataFrame
    input_df = pd.DataFrame([input_data])
    
    # 确保所有特征都存在于输入数据中
    for feature in feature_names:
        if feature not in input_df.columns:
            input_df[feature] = 0
    
    # 只选择模型使用的特征
    return input_df[feature_names]

def predict_likes(model, input_data, feature_names):
    """使用模型预测点赞数"""
    # 准备输入数据
    X = prepare_input_for_prediction(input_data, feature_names)
    
    # 进行预测
    prediction = model.predict(X)[0]
    
    return max(0, round(prediction))  # 确保点赞数为非负整数

# 5. 创建UI界面
def create_ui():
    """创建用户界面"""
    # 创建选项卡
    tab = widgets.Tab()
    tab_load = widgets.VBox()
    tab_predict = widgets.VBox()
    
    # ===== 选项卡1: 加载数据集和训练模型 =====
    xlsx_files = glob.glob("*.xlsx")
    latest_file = max(xlsx_files, key=os.path.getmtime) if xlsx_files else ""
    file_path_input = widgets.Text(
        value=latest_file,
        description='文件路径:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    # 创建模型选择下拉框
    model_options = ['训练新模型'] + get_available_models()
    model_dropdown = widgets.Dropdown(
        options=model_options,
        value='训练新模型',
        description='选择模型:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    load_button = widgets.Button(
        description='加载并训练模型',
        button_style='primary',
        layout=widgets.Layout(width='200px')
    )

    filter_outliers = widgets.Checkbox(
    value=True,
    description='过滤异常点赞值',
    disabled=False
    )

    percentile_slider = widgets.IntSlider(
        value=99,
        min=90,
        max=99,
        step=1,
        description='过滤百分位:',
        disabled=False,
        style={'description_width': 'initial'}
    )

    model_output = widgets.Output()
    
    tab_load.children = [file_path_input, model_dropdown, 
                     widgets.HBox([filter_outliers, percentile_slider]),
                     load_button, model_output]
    
    # ===== 选项卡2: 手动输入数据进行预测 =====
    # 创建预测时的模型选择下拉框
    predict_model_dropdown = widgets.Dropdown(
        options=get_available_models() if get_available_models() else ['请先训练模型'],
        value=get_available_models()[0] if get_available_models() else '请先训练模型',
        description='选择模型:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    # 创建输入字段
    comment_text = widgets.Textarea(
        value='',
        placeholder='请输入评论内容',
        description='评论内容:',
        disabled=False,
        layout=widgets.Layout(width='500px', height='100px')
    )
    
    comment_length = widgets.IntText(
        value=0,
        description='评论字数:',
        disabled=False,
        layout=widgets.Layout(width='300px')
    )
    
    has_emoji = widgets.Dropdown(
        options=[('否', 0), ('是', 1)],
        value=0,
        description='带表情:',
        layout=widgets.Layout(width='200px')
    )
    
    source = widgets.Dropdown(
        options=[('B站', 0), ('小红书', 1)],
        value=0,
        description='来源:',
        layout=widgets.Layout(width='200px')
    )
    
    # B站特有字段
    time_slot = widgets.Dropdown(
        options=[('早', 0), ('中', 1), ('晚', 2)],
        value=2,
        description='发布时段:',
        layout=widgets.Layout(width='200px')
    )
    
    user_level = widgets.Dropdown(
        options=[('普通用户', 0), ('资深用户', 1)],
        value=0,
        description='用户等级:',
        layout=widgets.Layout(width='200px')
    )
    
    # 小红书特有字段
    ip_address = widgets.Text(
        value='',
        placeholder='如: 广东',
        description='IP地址:',
        disabled=False,
        layout=widgets.Layout(width='200px')
    )
    
    likes = widgets.IntText(
        value=0,
        description='实际点赞数(可选):',
        disabled=False,
        layout=widgets.Layout(width='300px')
    )
    
    predict_button = widgets.Button(
        description='预测点赞数',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )
    
    record_button = widgets.Button(
        description='记录本次预测',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    prediction_output = widgets.Output()
    
    # 创建来源特定字段的容器
    source_specific_container = widgets.HBox([time_slot, user_level])
    
    tab_predict.children = [
        predict_model_dropdown,
        widgets.HBox([comment_text]),
        widgets.HBox([comment_length, has_emoji]),
        widgets.HBox([source]),
        source_specific_container,
        widgets.HBox([likes]),
        widgets.HBox([predict_button, record_button]),
        prediction_output
    ]
    
    # 设置选项卡
    tab.children = [tab_load, tab_predict]
    tab.set_title(0, '训练模型')
    tab.set_title(1, '点赞预测')
    
    # 全局变量，用于存储模型和特征名称
    global_vars = {
        'model': None,
        'feature_names': None,
        'predictions': []  # 存储历史预测
    }

    # 处理来源变更的函数
    def on_source_change(change):
        if change['name'] == 'value':
            # 先清空容器
            source_specific_container.children = []
            
            # 根据选择的来源填充容器
            if change['new'] == 0:  # B站
                source_specific_container.children = [time_slot, user_level]
            else:  # 小红书
                source_specific_container.children = [ip_address]

    # 更新预测模型下拉菜单
    def update_predict_model_dropdown():
        models = get_available_models()
        if models:
            predict_model_dropdown.options = models
            predict_model_dropdown.value = models[0]
        else:
            predict_model_dropdown.options = ['请先训练模型']
            predict_model_dropdown.value = '请先训练模型'

    # 按钮事件处理
    def on_load_button_clicked(b):
        with model_output:
            clear_output()
            try:
                # 加载数据
                file_path = file_path_input.value.strip()
                
                # 检查文件路径是否为空
                if not file_path:
                    print("错误: 请提供有效的数据文件路径")
                    print("提示: 请在\"文件路径\"输入框中输入Excel文件的位置")
                    return
                
                df = load_data(file_path)
                
                # 检查是否成功加载数据
                if df is None:
                    return  # load_data函数已经显示了错误信息
                
                # 数据预处理
                df_processed = preprocess_data(df)
                
                # 确定是训练新模型还是继续训练已有模型
                existing_model = None
                if model_dropdown.value != '训练新模型':
                    print(f"加载已有模型: {model_dropdown.value}")
                    existing_model = load_model(model_dropdown.value)
                    if existing_model is None:
                        print("加载模型失败，将训练新模型")
                
                # 训练模型
                # 修改训练模型调用部分
                model, feature_importance = train_model(
                    df_processed, 
                    existing_model=existing_model,
                    filter_outliers=filter_outliers.value,
                    percentile_threshold=percentile_slider.value
                )
                
                # 保存模型和特征名称
                global_vars['model'] = model
                global_vars['feature_names'] = feature_importance['Feature'].tolist()
                
                # 保存模型到文件
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                
                if model_dropdown.value == '训练新模型':
                    model_filename = f"rf_model_{timestamp}.pkl"
                else:
                    # 如果继续训练已有模型，为文件名添加"_updated"后缀
                    base_name = os.path.splitext(model_dropdown.value)[0]
                    model_filename = f"{base_name}_updated_{timestamp}.pkl"
                
                model_path = os.path.join('models', model_filename)
                with open(model_path, 'wb') as f:
                    pickle.dump(model, f)
                print(f"\n模型已保存到 '{model_path}'")
                
                # 更新模型下拉菜单
                model_dropdown.options = ['训练新模型'] + get_available_models()
                update_predict_model_dropdown()
                
            except Exception as e:
                print("处理过程中出现错误:")
                print(f"- {str(e)}")
                print("\n请检查文件格式是否正确，并确保包含所有必要的特征列。")
                import traceback
                traceback.print_exc()
    
    def on_predict_button_clicked(b):
        with prediction_output:
            clear_output()
            try:
                # 检查是否有可用模型
                if predict_model_dropdown.value == '请先训练模型':
                    print("请先在'训练模型'标签页训练模型！")
                    return
                
                # 如果全局变量中没有模型，或选择了不同的模型，则加载选定的模型
                selected_model = predict_model_dropdown.value
                if global_vars['model'] is None or selected_model != getattr(global_vars.get('current_model_name', None), 'value', None):
                    print(f"加载模型: {selected_model}")
                    model = load_model(selected_model)
                    if model is None:
                        print("模型加载失败！")
                        return
                    
                    # 从模型中获取特征名称
                    feature_names = [f for f in model.feature_names_in_]
                    
                    # 更新全局变量
                    global_vars['model'] = model
                    global_vars['feature_names'] = feature_names
                    global_vars['current_model_name'] = selected_model
                
                # 收集输入数据
                input_data = {
                    'comment_length': comment_length.value,
                    'has_emoji_num': has_emoji.value,
                    'source_num': source.value,
                    'interaction_score': 0,  # 默认值
                    'reply_hour': datetime.now().hour,  # 当前小时
                    'reply_dayofweek': datetime.now().weekday()  # 当前星期几
                }
                
                # 根据来源添加特定字段
                if source.value == 0:  # B站
                    input_data['time_slot_num'] = time_slot.value
                    input_data['user_level_converted_num'] = user_level.value
                else:  # 小红书
                    # 小红书不需要时间段和用户等级，但可能需要IP
                    input_data['time_slot_num'] = -1  # 默认值
                    input_data['user_level_converted_num'] = -1  # 默认值
                
                # 预测点赞数
                prediction = predict_likes(global_vars['model'], input_data, global_vars['feature_names'])
                
                print(f"预测点赞数: {prediction}")
                
                # 存储预测结果
                prediction_result = {
                    'comment_text': comment_text.value,
                    'comment_length': comment_length.value,
                    'has_emoji': '是' if has_emoji.value == 1 else '否',
                    'source': 'B站' if source.value == 0 else '小红书',
                    'predicted_likes': prediction,
                    'actual_likes': likes.value if likes.value > 0 else None,
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'model_used': predict_model_dropdown.value
                }
                
                # 添加来源特定信息
                if source.value == 0:  # B站
                    prediction_result['time_slot'] = ['早', '中', '晚'][time_slot.value]
                    prediction_result['user_level'] = '资深用户' if user_level.value == 1 else '普通用户'
                else:  # 小红书
                    prediction_result['ip_address'] = ip_address.value
                
                global_vars['predictions'].append(prediction_result)
                
            except Exception as e:
                print(f"预测错误: {str(e)}")
                import traceback
                traceback.print_exc()
    
    def on_record_button_clicked(b):
        with prediction_output:
            clear_output()
            try:
                if not global_vars['predictions']:
                    print("没有可记录的预测结果！")
                    return
                
                # 获取最新一次预测
                latest_prediction = global_vars['predictions'][-1]
                
                # 如果提供了实际点赞数，更新记录
                if likes.value > 0:
                    latest_prediction['actual_likes'] = likes.value
                
                # 转换为DataFrame并保存
                predictions_df = pd.DataFrame(global_vars['predictions'])
                output_file = f"likes_predictions_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
                predictions_df.to_excel(output_file, index=False)
                
                print(f"预测记录已保存到: {output_file}")
                display(predictions_df.tail(5))  # 显示最近5次预测
                
            except Exception as e:
                print(f"记录错误: {str(e)}")
                import traceback
                traceback.print_exc()
    
    # 更新评论字数
    def update_comment_length(change):
        if change['type'] == 'change' and change['name'] == 'value':
            comment_length.value = len(change['new'])
    
    # 连接事件处理函数
    load_button.on_click(on_load_button_clicked)
    predict_button.on_click(on_predict_button_clicked)
    record_button.on_click(on_record_button_clicked)
    comment_text.observe(update_comment_length)
    source.observe(on_source_change)
    
    # 初始化界面
    on_source_change({'name': 'value', 'new': source.value})
    
    return tab

# 6. 主程序
def main():
    # 创建并显示UI
    ui = create_ui()
    display(ui)

# 运行主程序
main()